In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_percentage_error
# from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import RANSACRegressor, LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
# from pytorch_tabnet.tab_model import TabNetRegressor
# import torch
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel



In [25]:
import pandas as pd
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Define a function to get the trained model for each property
def get_trained_final_model(data, target, property_name):
    """
    Trains the best performing model for a specific blend property on the full training data.
    """

    # Enhanced Gaussian Process kernel
    def create_gp_model():
        kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2)) + WhiteKernel()
        return make_pipeline(StandardScaler(), GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-10, normalize_y=True, random_state=42))

    # Final model info
    final_model_info = {
        'BlendProperty1': ('Gaussian_Process', create_gp_model()),
        'BlendProperty2': ('Gaussian_Process', create_gp_model()),
        'BlendProperty3': ('ElasticNet', create_gp_model()),
        'BlendProperty4': ('Gaussian_Process', create_gp_model()),
        'BlendProperty5': ('Random_Forest', create_gp_model()),
        'BlendProperty6': ('Gaussian_Process', create_gp_model()),
        'BlendProperty7': ('SVR_Poly', create_gp_model()),
        'BlendProperty8': ('ElasticNet', create_gp_model()),
        'BlendProperty9': ('ElasticNet', create_gp_model()),
        'BlendProperty10': ('Neural_Network', Sequential([
            Dense(64, activation='relu', input_shape=(data.shape[1],)),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dense(1)
        ]))
    }

    model_name, model = final_model_info[property_name]
    X = data
    y = target

    print(f"Training {model_name} for {property_name} on full dataset...")

    if model_name == 'Neural_Network':
        model.compile(optimizer='adam', loss='mae')
        model.fit(X, y, epochs=100, batch_size=32, verbose=0)
    elif model_name == 'TabNet':
        X_np = X.values
        y_np = y.values.reshape(-1, 1)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_np)
        model.fit(X_scaled, y_np, max_epochs=200, patience=20, batch_size=256, virtual_batch_size=128, verbose=0)
        class TabNetPipeline:
            def __init__(self, scaler, tabnet_model):
                self.scaler = scaler
                self.tabnet_model = tabnet_model
            def predict(self, X):
                X_scaled = self.scaler.transform(X.values)
                return self.tabnet_model.predict(X_scaled).flatten()
        model = TabNetPipeline(scaler, model)
    elif isinstance(model, Pipeline):
        model.fit(X, y)
    else:
        model.fit(X, y)

    print(f"Training complete for {property_name}.")
    return model

# Load test data and sample submission
try:
    test_df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv")
    submission_df = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/sample_solution.csv")
    test_ids = test_df['ID']
    test_df_features = test_df.drop(columns=['ID'])
except FileNotFoundError:
    print("Make sure 'test.csv' and 'sample_solution.csv' are uploaded to your Colab session.")

# Prediction phase
if 'test_df_features' in locals():
    for i in range(1, 11):
        property_name = f'BlendProperty{i}'
        print(f"\nProcessing {property_name} for final submission...")

        # Define features for this property
        features = ['Component1_fraction', 'Component2_fraction', 'Component3_fraction',
                    'Component4_fraction', 'Component5_fraction'] + \
                   [f'Component{j}_Property{i}' for j in range(1, 6)]

        trained_model = get_trained_final_model(df[features], df[property_name], property_name)

        test_predictions = trained_model.predict(test_df_features[features])
        submission_df[property_name] = test_predictions

    # Save the submission
    submission_df.to_csv('final_model_submission.csv', index=False)
    print("\n" + "="*80)
    print("Final submission file 'final_model_submission.csv' created successfully.")
    print("="*80)



Processing BlendProperty1 for final submission...
Training Gaussian_Process for BlendProperty1 on full dataset...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:450: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the

Training complete for BlendProperty1.

Processing BlendProperty2 for final submission...
Training Gaussian_Process for BlendProperty2 on full dataset...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:440: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete for BlendProperty2.

Processing BlendProperty3 for final submission...
Training ElasticNet for BlendProperty3 on full dataset...
Training complete for BlendProperty3.

Processing BlendProperty4 for final submission...
Training Gaussian_Process for BlendProperty4 on full dataset...


...

Training complete for BlendProperty8.

Processing BlendProperty9 for final submission...
Training ElasticNet for BlendProperty9 on full dataset...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete for BlendProperty9.

Processing BlendProperty10 for final submission...
Training Neural_Network for BlendProperty10 on full dataset...


/Users/MacbookPro/LocalStorage/Developer/ShellAi/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training complete for BlendProperty10.
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Final submission file 'final_model_submission.csv' created successfully.
